**Goal: Variational Inference on BNP Mixture of Projected Gammas**

Model:
$$
\begin{aligned}
y_i\mid\alpha_i &\sim \text{PG}_p(Y_i\mid\alpha_i,1_d)\\
\log\alpha_i &\sim \mathcal{G}\\
G &\sim \mathcal{PY}(\alpha,d,G_0)\\
\end{aligned}
~\hspace{1cm}
\begin{aligned}
G_0 &= \mathcal{N}_d\left(\log\alpha\mid\mu,\Sigma\right)\\
\mu &\sim \mathcal{N}_d(\mu_0,I_d)\\
\Sigma &\sim \mathcal{IW}\left(\nu,\Psi\right)\\
\end{aligned}
$$

In [1]:
import silence_tensorflow.auto
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import bijectors as tfb
from numpy.random import gamma

from tfprojgamma import ProjectedGamma
from data import Data
# Set random seeds for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
def stickbreak(v):
    """
    Creates a probability vector (sums to 1) from the vector of independent betas
    grabbed from https://luiarthur.github.io/TuringBnpBenchmarks/dpsbgmm 
        kind of worried about numerical stability here... isn't cumprod going to induce a lot of floating point issues?
    """
    batch_ndims = len(v.shape) - 1
    cumprod_one_minus_v = tf.math.cumprod(1 - v, axis=-1)
    one_v = tf.pad(v, [[0, 0]] * batch_ndims + [[0, 1]], "CONSTANT", constant_values=1)
    c_one = tf.pad(cumprod_one_minus_v, [[0, 0]] * batch_ndims + [[1, 0]], "CONSTANT", constant_values=1)
    return one_v * c_one

In [ ]:
def generative_model(nDat, nCol, nClust, dtype = np.float32):
    return tfd.JointDistributionNamed(dict(
        mu = tfd.MultivariateNormalDiag(loc = np.zeros(nCol, dtype), scale_diag = np.ones(nCol, dtype))
        Sigma = tfd.TransformedDistribution
        
        logalpha = tfd.MultivariateNormalTriL(loc = mu, scale_tril = Sigma
        
        v = tfd.Independent(tfd.Beta(np.ones(nClust - 1, dtype))),
    ))
    
    

In [2]:
# Set up shape parameters
alpha_true = gamma(size = 5, shape = 1.5)
print(alpha_true)
dist1 = ProjectedGamma(alpha_true, 10)
Yp    = tf.cast(dist1.sample(200), tf.float32)

[3.94761907 0.62279234 0.68411215 0.34912511 4.2482095 ]


In [3]:
# prior shape parameters
log_alpha_0 = tf.ones(5, dtype = tf.float32) * np.log(0.5)

**Define the Joint Distribution**

In [8]:
# define generator
def generative_model(log_alpha_0, n_samples):
    log_alpha = yield tfd.JointDistributionCoroutine.Root(
        tfd.MultivariateNormalDiag(
            loc = log_alpha_0, scale_diag = tf.ones(5, dtype = tf.float32), name = 'log_alpha'
            ),
        )
    Yp = yield tfd.Sample(
        ProjectedGamma(concentration = tf.exp(log_alpha) * tf.ones((n_samples, 5), dtype = tf.float32)),
        name = 'Yp',
        )

model_joint = tfd.JointDistributionCoroutineAutoBatched(
    lambda: generative_model(log_alpha_0, 200),
    )

model_joint_log_prob = lambda log_alpha: model_joint.log_prob(log_alpha, Yp)

**Verifying the structure of the joint model**

In [9]:
model_joint

<tfp.distributions.JointDistributionCoroutineAutoBatched 'JointDistributionCoroutineAutoBatched' batch_shape=[] event_shape=StructTuple(
  log_alpha=[5],
  Yp=[200, 5]
) dtype=StructTuple(
  log_alpha=float32,
  Yp=float32
)>

**Mean Field Variational Bayes -- Independence between columns**
$$
\log\alpha \sim \prod_{\ell = 1}^d\text{Normal}(\log\alpha_{\ell} \mid \mu_{q\ell}, \sigma_{q\ell})
$$

In [10]:
q_mu = tf.Variable(log_alpha_0, dtype = tf.float32)
q_scale = tfp.util.TransformedVariable(np.ones(5), tfb.Exp(), dtype = tf.float32)

surrogate_posterior = tfd.MultivariateNormalDiag(loc = q_mu, scale_diag = q_scale, name = 'surrogate 1')

with tf.GradientTape() as g:
    samples = surrogate_posterior.sample(100)
    neg_elbo = -tf.reduce_mean(model_joint_log_prob(samples) - surrogate_posterior.log_prob(samples))
print(g.gradient(neg_elbo, surrogate_posterior.trainable_variables)) # exists!

(<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-118.478775,  174.52089 ,  197.55557 ,  481.3123  , -154.02953 ],
      dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([151.6519  , 276.73755 , 280.07654 , 725.90686 ,  67.493996],
      dtype=float32)>)


In [16]:
path = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn = model_joint_log_prob,
    surrogate_posterior = surrogate_posterior,
    optimizer = tf.optimizers.Adam(.2),
    num_steps = 1000,
    sample_size = 500,
    )

print(tf.exp(q_mu)) # This appears to have worked; the values end in *rougly* the right place.
print(q_scale**2)

tf.Tensor([3.9468846 0.597943  0.6775192 0.3909855 4.0545163], shape=(5,), dtype=float32)
tf.Tensor([0.00176153 0.00362453 0.00339159 0.00389452 0.00167895], shape=(5,), dtype=float32)


**Gaussian Variational Bayes -- Dependence Between Columns**
$$
\log\alpha \sim \text{MVNormal}(\log\alpha \mid \mu_q, \Sigma_q)
$$

In [12]:
# New Style: Make the variational Parameters
q_nu = tf.Variable(tf.zeros(5, dtype = tf.float32), name = 'Mu Surrogate (mean of log alpha)')
cholbijector = tfb.FillScaleTriL(diag_bijector = tfb.Exp())
q_Lu = tfp.util.TransformedVariable(tf.eye(5), bijector = cholbijector)

surrogate_posterior_mvnorm = tfd.MultivariateNormalTriL(loc = q_nu, scale_tril = q_Lu)

with tf.GradientTape() as g:
    samples = surrogate_posterior_mvnorm.sample(100)
    neg_elbo = -tf.reduce_mean(model_joint_log_prob(samples) - surrogate_posterior_mvnorm.log_prob(samples))
print(g.gradient(neg_elbo, surrogate_posterior_mvnorm.trainable_variables)) # Exists!

(<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-153.65451,  987.5581 ,  499.819  , 1076.0623 , -185.46274],
      dtype=float32)>, <tf.Tensor: shape=(15,), dtype=float32, numpy=
array([ 208.05101 ,  -68.88415 ,   23.579296,  -62.816746, -142.34943 ,
        192.73282 , 1257.155   , -517.5065  ,  -53.48645 ,  140.62529 ,
       -450.41083 , 2185.321   ,  676.94464 ,  -16.753181,  -39.66961 ],
      dtype=float32)>)


In [13]:
path_mvnorm = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn = model_joint_log_prob,
    surrogate_posterior = surrogate_posterior_mvnorm,
    optimizer = tf.optimizers.Adam(.2),
    num_steps = 1000,
    sample_size = 500,
    )
print(tf.exp(q_nu)) # This gives the same basic response as previous.

tf.Tensor([3.9388332  0.59376836 0.67372936 0.38574114 4.06507   ], shape=(5,), dtype=float32)


In [15]:
(q_Lu.numpy() @ q_Lu.numpy().T < 0)

array([[0.00403725, 0.00055345, 0.00169735, 0.00089325, 0.00248185],
       [0.00055345, 0.00411207, 0.00131955, 0.000444  , 0.00097888],
       [0.00169735, 0.00131955, 0.0048471 , 0.00037286, 0.00064957],
       [0.00089325, 0.000444  , 0.00037286, 0.00465075, 0.00178326],
       [0.00248185, 0.00097888, 0.00064957, 0.00178326, 0.00381977]],
      dtype=float32)

I guess it makes some sense that the posterior covariance between parameters of the Dirichlet would be positive, despite the covariance between *values* of the Dirichlet being negative.